[![Fixel Algorithms](https://fixelalgorithms.co/images/CCExt.png)](https://fixelalgorithms.gitlab.io/)

# UnSupervised Learning Methods

## Dimensionality Reduction - Principal Component Analysis (PCA)

> Notebook by:
> - Royi Avital RoyiAvital@fixelalgorithms.com

## Revision History

| Version | Date       | User        |Content / Changes                                                   |
|---------|------------|-------------|--------------------------------------------------------------------|
| 0.1.000 | 02/05/2023 | Royi Avital | First version                                                      |
|         |            |             |                                                                    |

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/FixelAlgorithmsTeam/FixelCourses/blob/master/UnSupervisedLearningMethods/2023_03/0010DimensionalityReductionPCA.ipynb)

In [ ]:
# Import Packages

# General Tools
import numpy as np
import scipy as sp
import pandas as pd

# Machine Learning
from sklearn.datasets import load_breast_cancer
from sklearn.decomposition import PCA
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Miscellaneous
import os
import math
from platform import python_version
import random

# Typing
from typing import Callable, List, Tuple, Union

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Jupyter
from IPython import get_ipython
from IPython.display import Image, display
from ipywidgets import Dropdown, FloatSlider, interact, IntSlider, Layout

## Notations

* <font color='red'>(**?**)</font> Question to answer interactively.
* <font color='blue'>(**!**)</font> Simple task to add code for the notebook.
* <font color='green'>(**@**)</font> Optional / Extra self practice.
* <font color='brown'>(**#**)</font> Note / Useful resource / Food for thought.

In [ ]:
# Configuration
%matplotlib inline

seedNum = 512
np.random.seed(seedNum)
random.seed(seedNum)

# sns.set_theme() #>! Apply SeaBorn theme

runInGoogleColab = 'google.colab' in str(get_ipython())

In [ ]:
# Constants

FIG_SIZE_DEF    = (8, 8)
ELM_SIZE_DEF    = 50
CLASS_COLOR     = ('b', 'r')
EDGE_COLOR      = 'k'
MARKER_SIZE_DEF = 10
LINE_WIDTH_DEF  = 2


In [ ]:
# Fixel Algorithms Packages


## Dimensionality Reduction by PCA

In this exercise we'll use the PCA approach for dimensionality reduction within a pipeline.

This exercise introduces:

1. Working with the [Breast Cancer Wisconsin Data Set](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html).
1. Combine the PCA as a transformer in a pipeline with a linear classifier to predict the binary class of the data.  
2. Select the best features using a sequential approach.

The objective is to optimize the feature selection in order to get the best classification accuracy.


* <font color='brown'>(**#**)</font> PCA is the most basic dimensionality reduction operator.
* <font color='brown'>(**#**)</font> The PCA output is a linear combination of the input.

In [ ]:
# Parameters

# Model
numComp  = 2
paramC   = 1
numKFold = 5
numFeat  = 5



In [ ]:
# Auxiliary Functions

hOrdinalNum = lambda n: '%d%s' % (n, 'tsnrhtdd'[(((math.floor(n / 10) %10) != 1) * ((n % 10) < 4) * (n % 10))::4])

def PlotScatterData(mX: np.ndarray, vL: np.ndarray, hA:plt.Axes = None, figSize: Tuple[int, int] = FIG_SIZE_DEF, markerSize: int = MARKER_SIZE_DEF, lineWidth: int = LINE_WIDTH_DEF, axisTitle: str = None):

    if hA is None:
        hF, hA = plt.subplots(figsize = figSize)
    else:
        hF = hA.get_figure()
    
    vU = np.unique(vL)
    numClusters = len(vU)

    for ii in range(numClusters):
        vIdx = vL == vU[ii]
        hA.scatter(mX[vIdx, 0], mX[vIdx, 1], s = ELM_SIZE_DEF, edgecolor = EDGE_COLOR, label = ii)
    
    hA.set_xlabel('${{x}}_{{1}}$')
    hA.set_ylabel('${{x}}_{{2}}$')
    if axisTitle is not None:
        hA.set_title(axisTitle)
    hA.grid()
    hA.legend()

    # return hF


## Generate / Load Data

In this notebook we'll use the [Breast Cancer Wisconsin Data Set](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html).

Features are computed from a digitized image of a fine needle aspirate (FNA) of a breast mass.

![](https://i.imgur.com/4LE2biE.png)


In [ ]:
# Loading / Generating Data

mX, vY   = load_breast_cancer(return_X_y = True)
dfX, dsY = load_breast_cancer(return_X_y = True, as_frame = True)


print(f'The features data shape: {mX.shape}')

In [ ]:
# Add Label Data
dfData = dfX.copy()
dfData['Label'] = pd.Categorical(dsY)

dfData

## Exploratory Data Analysis (EDA)

#### Correlation Matrix

In [ ]:
# Correlation Matrix
hF, hA = plt.subplots(figsize = (14, 14))
dfData['Label'] = pd.to_numeric(dfData['Label'])
mC = dfData.corr(method = 'pearson')
sns.heatmap(mC.abs(), cmap = 'coolwarm', annot = True, fmt = '2.1f', ax = hA)

plt.show()

* <font color='red'>(**?**)</font> Are there redundant features? Think in the context of PCA.

## Pre Processing

In [ ]:
# Zero Mean, Unit Variance

oStdScaler  = StandardScaler()
mX          = oStdScaler.fit_transform(mX)

## Applying Dimensionality Reduction - PCA 

In [ ]:
# Applying the PCA Model

oPCA = PCA(n_components = numComp)
mZ   = oPCA.fit_transform(mX)

### Plot Data in 2D

In [ ]:
# Plot the 2D Result

PlotScatterData(mZ, vY)

* <font color='brown'>(**#**)</font> The _optimal_ Dimensionality Reduction is the perfect feature engineering.
* <font color='brown'>(**#**)</font> Dimensionality Reduction is usually used as a step in pipeline.
* <font color='red'>(**?**)</font> Can we use _Clustering_ as a dimensionality reduction?

## Pipeline with PCA

In this section we'll build a simple pipeline:

 - Apply `PCA` with 2 components.
 - Apply Linear Classifier.

We'll tweak the model with selecting the best features as an input to the `PCA`.  
To do that we'll use th [`SequentialFeatureSelector`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SequentialFeatureSelector.html#sklearn.feature_selection.SequentialFeatureSelector) object of SciKit Learn.  

Selecting features sequentially is a compute intensive operation.  
Hence we can use when the following assumptions hold:

1. The number of features is modest (< 100).
2. The cross validation loop (The estimator / pipeline `fit()` and `predict()`) process is fast.

Of course the time budget and computing budget are also main factors.


In [ ]:
# Building the Pipeline

oPipeCls = Pipeline([('PCA', PCA(n_components = numComp)), ('Classifier', LogisticRegression(C = paramC))])

In [ ]:
# Base Line Score

scoreAccBase = np.mean(cross_val_score(oPipeCls, mX, vY, cv = numKFold))

* <font color='red'>(**?**)</font> What are the issues with `cross_val_score`? Think the cases where folds are not evenly divided or the score is not linear.

In [ ]:
# Selecting the Features

oFeatSelector = SequentialFeatureSelector(estimator = oPipeCls, n_features_to_select = numFeat, direction = 'backward', cv = numKFold)
oFeatSelector = oFeatSelector.fit(mX, vY)

In [ ]:
# Extracting Selected Features
vSelectedFeat = oFeatSelector.get_support()

* <font color='red'>(**?**)</font> How should we use the above results in production?

In [ ]:
# Optimized Score

scoreAccOpt = np.mean(cross_val_score(oPipeCls, mX[:, vSelectedFeat], vY, cv = numKFold))

In [ ]:
# Comparing Results

print(f'The base line score (Accuracy): {scoreAccBase:0.2%}.')
print(f'The optimized score (Accuracy): {scoreAccOpt:0.2%}.')

In [ ]:
# The Selected Features

dfX.columns[vSelectedFeat]

* <font color='red'>(**?**)</font> Look at the correlation matrix, how correlated are the selected features relative to other?
* <font color='red'>(**?**)</font> Given the pipeline above, can we think on a more efficient way to select features?
* <font color='green'>(**@**)</font> Optimize all hyper parameters of the model: `n_features_to_select`, `n_components` and `C`.
* <font color='brown'>(**#**)</font> One motivation to use PCA is to lower dimensionality with almost zero loss of information in order to accelerate algorithms which their complexity depends on the dimensionality (`t-SNE`).